# Titanic Survival Classificaion:


In [ ]:
#required libraries:
import pandas as pd;    
import numpy as np;

In [ ]:
#importing training and test data into pandas dataframe
def import_data(path):
    return pd.read_csv(path)



In [ ]:
#utility functions
def print_info(df):
    return df.info()

def print_description(df):
    return df.describe()

def print_head(df, count=5):
    return df.head(count)

In [211]:
titanic_train = import_data("./../../../Datasets/Titanic/train.csv");
titanic_test =  import_data("./../../../Datasets/Titanic/test.csv")

### Preprocess both training and testing dataframe :

In [212]:
def numeric_mapper(df, column_name):
    mapper = {}
    data_list = df[column_name].unique()
    data_list = ['missing' if pd.isnull(x) else x for x in data_list]
    data_list.sort()
    for i in range(0, len(data_list)):
        if data_list[i] == 'missing':
            mapper[np.nan] = 404
        else:    
            mapper[data_list[i]] = i
    return mapper
    
def data_preprocessor(df):
 
    df = (df.rename({'SibSp' : '# of Siblings', 
                                             'Parch': '# of Parents', 
                                             'Sex' : 'Gender',
                                             'Pclass' : 'Class'},
                                            axis = 1)
                                    .drop(['Name', 'Ticket', 'PassengerId'], axis = 1)
                                    .astype({'Gender' : pd.api.types.CategoricalDtype(df['Sex'].unique(), ordered=False), 
                                             'Class' : pd.api.types.CategoricalDtype(df['Pclass'].unique(), ordered=True)})
#                                     .replace({'Embarked' : {np.NaN : 'un-known'}})
                                    .replace({'Gender' : numeric_mapper(df, 'Sex'),
                                              'Embarked' : numeric_mapper(df, 'Embarked'),
                                              'Cabin' : numeric_mapper(df, 'Cabin')})
    #                                 .loc[:]
                      )
    return df

In [213]:
# preprocessing phase 1 for training and validation dataset
titanic_train = data_preprocessor(titanic_train)
print_head(titanic_train)

,Survived,Class,Gender,Age,# of Siblings,# of Parents,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,404,2
1,1,1,0,38.0,1,0,71.2833,81,0
2,1,3,0,26.0,0,0,7.9250,404,2
3,1,1,0,35.0,1,0,53.1000,55,2
4,0,3,1,35.0,0,0,8.0500,404,2


In [214]:
titanic_test = data_preprocessor(titanic_test)
print_head(titanic_test)

,Class,Gender,Age,# of Siblings,# of Parents,Fare,Cabin,Embarked
0,3,1,34.5,0,0,7.8292,404,1
1,3,0,47.0,1,0,7.0000,404,2
2,2,1,62.0,0,0,9.6875,404,1
3,3,1,27.0,0,0,8.6625,404,2
4,3,0,22.0,1,1,12.2875,404,2


### Dealing With Null Values :

#### CASE 1 : Removing attributes with null values:

In [215]:
# In training only 2 attributes contains a major chunk of null data : Age and Cabin
def drop_null_attributes(df):
    df = (df.dropna(axis='columns')
         )
    return df

#### CASE 2: Remove rows with null values:

In [216]:
def drop_null_rows(df):
    df = (df.dropna(axis='rows')
         )
    return df

### Classification Models :

#### Logistic Regression 

In [217]:
def create_logistic_regression_model(df):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression

    X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns!= 'Survived'], df['Survived'], test_size = 0.75, random_state = 0)
    clf = LogisticRegression(solver = 'liblinear') #solver : liblinear good for small datasets
    clf.fit(X_train, y_train)
    return clf.score(X_test, y_test)

In [218]:
titanic_train_1 = drop_null_attributes(titanic_train)
create_logistic_regression_model(titanic_train_1)

0.7772795216741405

In [220]:
titanic_train_2 = titanic_train.replace({'Cabin': {404 : np.nan}})
titanic_train_2 = drop_null_rows(titanic_train_2)
create_logistic_regression_model(titanic_train_2)

0.7194244604316546